# Data Analysis Pipeline Example

This notebook demonstrates the complete data analysis pipeline using the `data_analysis` package.

## Pipeline Steps:
1. **Load Data** - Import data from various formats (CSV, Excel, JSON)
2. **Clean Data** - Handle missing values, duplicates, and outliers
3. **Analyze Data** - Perform statistical analysis and correlation studies
4. **Visualize Data** - Create meaningful visualizations

In [ ]:
# Import required modules
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

from data_analysis import DataLoader, DataCleaner, DataAnalyzer, Visualizer

# Set up paths
data_dir = Path('../data/raw')
output_dir = Path('../data/processed')
output_dir.mkdir(parents=True, exist_ok=True)

# Configure matplotlib for inline plotting
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

## 1. Load Data

We'll load employee data from a CSV file.

In [ ]:
# Initialize DataLoader
loader = DataLoader()

# Load employee data
employees_df = loader.load_csv(data_dir / 'employees.csv')

print(f"Loaded {len(employees_df)} employee records")
print(f"\nColumns: {list(employees_df.columns)}")
print(f"\nFirst few rows:")
employees_df.head()

## 2. Data Inspection

Let's examine the data types and basic statistics.

In [ ]:
# Check data types
print("Data types:")
print(employees_df.dtypes)
print("\nBasic statistics:")
employees_df.describe()

## 3. Clean Data

Convert data types and check for missing values or duplicates.

In [ ]:
# Initialize DataCleaner
cleaner = DataCleaner(employees_df)

# Convert hire_date to datetime
cleaner.convert_dtypes({'hire_date': 'datetime64[ns]'})

# Check for missing values
missing = cleaner.df.isnull().sum()
print("Missing values per column:")
print(missing[missing > 0] if missing.sum() > 0 else "No missing values found")

# Check for duplicates
duplicates = cleaner.df.duplicated().sum()
print(f"\nDuplicate rows: {duplicates}")

# Get cleaned data
clean_df = cleaner.get_data()
print(f"\nCleaned data shape: {clean_df.shape}")

## 4. Statistical Analysis

Perform various statistical analyses on the cleaned data.

In [ ]:
# Initialize DataAnalyzer
analyzer = DataAnalyzer(clean_df)

# Get summary statistics
print("=== Summary Statistics ===")
summary = analyzer.get_summary_statistics()
display(summary)

# Analyze by department
print("\n=== Analysis by Department ===")
dept_analysis = analyzer.group_analysis(
    group_column='department',
    agg_columns=['salary', 'age', 'performance_score'],
    agg_funcs=['mean', 'median', 'std', 'count']
)
display(dept_analysis)

# Get correlation matrix for numeric columns
print("\n=== Correlation Analysis ===")
numeric_cols = ['age', 'salary', 'performance_score']
correlations = analyzer.get_correlation_matrix(columns=numeric_cols)
display(correlations)

## 5. Visualizations

Create various visualizations to understand the data better.

In [ ]:
# Initialize Visualizer
viz = Visualizer(clean_df)

# Create salary distribution histogram
print("Creating salary distribution histogram...")
viz.create_histogram(
    column='salary',
    bins=10,
    title='Salary Distribution',
    xlabel='Salary ($)',
    ylabel='Frequency',
    kde=True
)
plt.show()

In [ ]:
# Create boxplot for salary by department
print("Creating salary by department boxplot...")
viz.create_boxplot(
    column='salary',
    title='Salary Distribution by Department',
    ylabel='Salary ($)',
    groupby='department'
)
plt.show()

In [ ]:
# Create scatter plot of age vs salary
print("Creating age vs salary scatter plot...")
viz.create_scatter(
    x='age',
    y='salary',
    title='Age vs Salary',
    xlabel='Age',
    ylabel='Salary ($)',
    hue='department'
)
plt.show()

In [ ]:
# Create correlation heatmap
print("Creating correlation heatmap...")
viz.create_correlation_heatmap(
    columns=numeric_cols,
    title='Correlation Matrix: Age, Salary, and Performance',
    annot=True,
    cmap='coolwarm'
)
plt.show()

In [ ]:
# Create bar plot of average salary by department
print("Creating average salary by department bar plot...")
dept_salaries = clean_df.groupby('department')['salary'].mean().reset_index()
viz_dept = Visualizer(dept_salaries)
viz_dept.create_bar_plot(
    x='department',
    y='salary',
    title='Average Salary by Department',
    xlabel='Department',
    ylabel='Average Salary ($)'
)
plt.show()

## 6. Advanced Analysis

Perform regression analysis and outlier detection.

In [ ]:
# Linear regression: predict salary based on age
print("=== Linear Regression: Salary ~ Age ===")
slope, intercept, r_squared = analyzer.simple_linear_regression(x='age', y='salary')
print(f"Slope: {slope:.2f}")
print(f"Intercept: {intercept:.2f}")
print(f"R² Score: {r_squared:.4f}")
print(f"\nInterpretation: For each additional year of age, salary increases by ${slope:.2f}")

# Detect salary outliers
print("\n=== Outlier Detection (Salary) ===")
outliers_zscore = analyzer.detect_anomalies(column='salary', method='zscore', threshold=2.0)
outliers_iqr = analyzer.detect_anomalies(column='salary', method='iqr')

print(f"Outliers detected (Z-score method): {len(outliers_zscore)}")
if len(outliers_zscore) > 0:
    print("Outlier records:")
    display(clean_df.loc[outliers_zscore, ['name', 'department', 'salary']])

print(f"\nOutliers detected (IQR method): {len(outliers_iqr)}")
if len(outliers_iqr) > 0:
    print("Outlier records:")
    display(clean_df.loc[outliers_iqr, ['name', 'department', 'salary']])

## 7. Save Results

Save the cleaned data and analysis results.

In [ ]:
# Save cleaned data
output_file = output_dir / 'employees_cleaned.csv'
loader.save_csv(clean_df, output_file)
print(f"Cleaned data saved to: {output_file}")

# Save department analysis
dept_file = output_dir / 'department_analysis.csv'
loader.save_csv(dept_analysis, dept_file)
print(f"Department analysis saved to: {dept_file}")

# Save visualizations
viz_dir = Path('../outputs/visualizations')
viz_dir.mkdir(parents=True, exist_ok=True)

viz.create_histogram('salary', bins=10, kde=True, save_path=viz_dir / 'salary_dist.png')
viz.create_correlation_heatmap(numeric_cols, annot=True, save_path=viz_dir / 'correlation_heatmap.png')
print(f"\nVisualizations saved to: {viz_dir}")

## Summary

This notebook demonstrated a complete data analysis pipeline:

1. ✅ **Data Loading** - Loaded employee data from CSV
2. ✅ **Data Cleaning** - Converted data types and verified data quality
3. ✅ **Statistical Analysis** - Summary statistics, group analysis, and correlations
4. ✅ **Visualization** - Multiple plot types to understand the data
5. ✅ **Advanced Analysis** - Regression and outlier detection
6. ✅ **Results Export** - Saved cleaned data and visualizations

### Key Findings:
- The dataset contains 20 employees across 4 departments
- Salary correlates positively with age and performance score
- Management positions have the highest average salaries
- No missing values or duplicates were found

### Next Steps:
- Try loading the JSON sales data (`sales_data.json`)
- Experiment with different cleaning strategies
- Perform time-series analysis on hire dates
- Create custom visualizations for specific insights